In [1]:
from policyengine_us import Microsimulation

In [2]:
import os
token = os.environ["POLICYENGINE_GITHUB_MICRODATA_AUTH_TOKEN"]

In [3]:
sim = Microsimulation()

In [4]:
from policyengine_us.data.datasets.cps.enhanced_cps.loss import generate_model_variables
(
    household_weights,
    weight_adjustment,
    values_df,
    targets,
    targets_array,
    equivalisation_factors_array
) = generate_model_variables("cps_2021", 2025)

#This returns a set of household weights, a random tensor of the same size as the weights tensor,
#a Pandas dataframe to transform weights into statistical predictions, a dictionary of target values,
#an array of target values, and some equivalisation factors I don't understand.

In [12]:
type(weight_adjustment)

numpy.ndarray

In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

# Then we're working with: this new array * the weights = our estimate.
# Then our error in a prediction is based on |predicted - actual|/equivalisation factor. Square that to get
# square error, and then average to get MSE.

In [6]:
from reweight.logic import reweight

In [7]:
sim_matrix = torch.tensor(values_df.to_numpy(), dtype=torch.float32)
#FIXME: These inputs can't just be NumPy arrays! Also, fix reweight to accommodate stuff.
reweight(torch.tensor(household_weights, dtype=torch.float32), sim_matrix, targets, torch.tensor(targets_array, dtype=torch.float32))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
def calibrate(household_weights, weight_adjustment, values_df, targets, targets_array, equivalisation_factors_array):
    # Initialize a TensorBoard writer
    writer = SummaryWriter()

    #Create a Torch tensor of log weights
    log_weights = torch.log(household_weights)
    log_weights.requires_grad_()

    sim_matrix = values_df.to_numpy()

    # sim_matrix (cross) exp(log_weights) = targets_array
    sim_matrix = torch.tensor(sim_matrix, dtype=torch.float32)
    #targets_array will be our target values.

    optimizer = torch.optim.Adam([log_weights])

    # Training loop
    num_epochs = 1000
    for epoch in range(num_epochs):

        # Estimate the targets
        targets_estimate = torch.exp(log_weights) @ sim_matrix
        # Calculate the loss
        loss = torch.mean(((targets_estimate - targets_array)/equivalisation_factors_array) ** 2)

        writer.add_scalar("Loss/train", loss, epoch)

        optimizer.zero_grad()

        # Perform backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        # Print loss for every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    writer.flush()

    final_weights = np.exp(log_weights.detach().numpy())
    final_estimates = (
        final_weights @ sim_matrix.numpy()
    )
    true_values = targets
    #print("Final weights:", final_weights)
    #print("Final estimates:", final_estimates)
    #print("True values:", true_values)

In [ ]:
calibrate(household_weights,
    weight_adjustment,
    values_df,
    targets,
    targets_array,
    equivalisation_factors_array)

Epoch 0, Loss: 0.2274675965309143
Epoch 100, Loss: 0.18678854405879974
Epoch 200, Loss: 0.15837892889976501
Epoch 300, Loss: 0.13632304966449738
Epoch 400, Loss: 0.11881797015666962
Epoch 500, Loss: 0.1046074628829956
Epoch 600, Loss: 0.09283030778169632
Epoch 700, Loss: 0.08289289474487305
Epoch 800, Loss: 0.0743781179189682
Epoch 900, Loss: 0.06698659062385559
